In [1]:
import os
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
os.environ["KERAS_BACKEND"] = "torch"
import keras
from datamodules import data_processors

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
datos_combi  = pd.read_csv(path_archivo)
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [3]:
"""
Importamos el modelo 
"""
path_modelo = "models/isotropico_act_custom_6.keras"
n_pow = 20
def custom_activation(x):
    return (1/n_pow) * torch.log((1 + torch.exp(n_pow * x)) / (1 + torch.exp(n_pow * (x - 1))))
#fin función

@keras.saving.register_keras_serializable()
class CustomActivationLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomActivationLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        # Ensure inputs are Torch tensors
        inputs = torch.tensor(inputs) if not isinstance(inputs, torch.Tensor) else inputs
        return custom_activation(inputs)
    #fin función
#fin clase
modelo = keras.models.load_model(path_modelo,  custom_objects={"CustomActivationLayer": CustomActivationLayer})

In [4]:
num_layers = len(modelo.layers)
print("El modelo tiene " + str(num_layers) + " capas")

El modelo tiene 5 capas


In [5]:
for i, layer in enumerate(modelo.layers):
    if hasattr(layer, 'units'):
        print(f"Layer {i + 1}: {layer.name} has {layer.units} neurons.")

Layer 1: dense has 64 neurons.
Layer 2: dense_1 has 32 neurons.
Layer 3: dense_2 has 8 neurons.
Layer 4: dense_3 has 1 neurons.


In [10]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "nn/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "eig_" + str(x+1), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred/" + nombre
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "red_neuronal/", nombre_feat = "l_moño", N_phi_K = 500):
    N_datos = len(d_frame)
    N_partes = int(N_datos/N_phi_K)
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función

In [7]:
N_eig = 5
features = ["eta", "beta"] + list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
target = "phi_K"
datos_combi = data_processors.preprocess_data(datos_combi, N_eig, target)
X_combi = datos_combi[features]
y_combi = datos_combi[target]
y_gorro = modelo.predict(X_combi)

1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [8]:
mets_combi = data_processors.get_metrics(X_combi, y_combi, modelo)
print("Mis datos combinatoriales: ", mets_combi)

1641/1641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Mis datos combinatoriales:  {'R2': 0.9802012233494914, 'RMSE': np.float64(0.04056703472395429), 'MAE': np.float64(0.027278970722211474)}


In [11]:
graficar_todas(datos_combi, 3, target, preds={"y": y_gorro}, nombre = "red_neuronal_6/")